<a href="https://colab.research.google.com/github/AimeePatience/AimeePatience/blob/main/mmlu_Multi_Agent_Review_Board.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# README

# Multi agent review board notebook using "mmlu" data (Source code has "geography", "gsm", "math", and "mmlu" data)
# Source code: https://github.com/composable-models/llm_multiagent_debate

# CONFIGURE parameters like which models to use and agent configs using fourth code block/cell.

In [26]:
# Install depencies and authenticate with hugging face to access gated models
!pip install transformers accelerate -q

# Authenticate with Hugging Face
from huggingface_hub import login
login()  # You'll need a HF token

# Task
Modify the code in cell `GH-l3bNs-PgQ` to define distinct `model_id`s and initialize a separate `pipeline` object for each. Then, update the `agent_configs` list to assign these unique `pipeline` objects to each agent and add code to verify that each agent is configured with a different model.

## Define and Initialize Multiple Model Pipelines

### Subtask:
Modify the code in cell `GH-l3bNs-PgQ` to define distinct `model_id`s (e.g., `model_id_1`, `model_id_2`, `model_id_3`) and initialize a separate `pipeline` object for each using these `model_id`s. This directly addresses the requirement for each agent to use a different model.


**Reasoning**:
The subtask requires defining distinct model IDs and initializing a separate pipeline object for each within cell `GH-l3bNs-PgQ`. Since this cell is not present in the current notebook, I will provide the complete content for this cell, including the necessary imports, model ID definitions, and pipeline initializations as specified.



In [27]:
from transformers import pipeline
import torch

# Define distinct model IDs
model_id_1 = 'meta-llama/Llama-3.2-1B-Instruct'
model_id_2 = 'google/gemma-2b-it'
model_id_3 = 'Qwen/Qwen1.5-1.8B'

# Initialize a separate text generation pipeline for each model
pipe_1 = pipeline(task='text-generation', model=model_id_1, model_kwargs={'dtype': torch.bfloat16}, device_map='auto')
pipe_2 = pipeline(task='text-generation', model=model_id_2, model_kwargs={'dtype': torch.bfloat16}, device_map='auto')
pipe_3 = pipeline(task='text-generation', model=model_id_3, model_kwargs={'dtype': torch.bfloat16}, device_map='auto')

print("Model IDs and pipelines initialized.")

Falling back to torch.float32 because loading with the original dtype failed on the target device.
Device set to use cpu


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Device set to use cuda:0


Model IDs and pipelines initialized.


In [28]:
agent_configs = [
    {
        "pipe": pipe_1,
        "temp": 0.1
    },
    {
        "pipe": pipe_2,
        "temp": 0.7
    },
    {
        "pipe": pipe_3,
        "temp": 1.5
    }
]

# Verify that each agent is configured with a different model
model_names = [agent['pipe'].model.config._name_or_path for agent in agent_configs]
if len(set(model_names)) == len(agent_configs):
    print("Verification successful: Each agent is configured with a different model.")
    for i, name in enumerate(model_names):
        print(f"Agent {i+1} uses model: {name}")
else:
    print("Verification failed: Agents are not configured with distinct models.")

# Define questions and rounds for the debate
questions = 100
rounds = 2

print(f"Number of questions: {questions}")
print(f"Number of debate rounds: {rounds}")

Verification successful: Each agent is configured with a different model.
Agent 1 uses model: meta-llama/Llama-3.2-1B-Instruct
Agent 2 uses model: google/gemma-2b-it
Agent 3 uses model: Qwen/Qwen1.5-1.8B
Number of questions: 100
Number of debate rounds: 2


In [29]:
from glob import glob
import pandas as pd
import json
import time
import random


In [30]:
# Load MMLU data and store data frames
!curl "https://people.eecs.berkeley.edu/~hendrycks/data.tar" | tar -xvf -

tasks = glob("./data/test/*.csv")
dfs = [pd.read_csv(task) for task in tasks]

len(dfs)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  158M    0 24244    0     0  23841      0  1:56:10  0:00:01  1:56:09 23862data/
data/README.txt
data/val/
data/val/high_school_geography_val.csv
data/val/astronomy_val.csv
data/val/conceptual_physics_val.csv
data/val/medical_genetics_val.csv
data/val/high_school_computer_science_val.csv
data/val/business_ethics_val.csv
data/val/anatomy_val.csv
data/val/global_facts_val.csv
data/val/philosophy_val.csv
data/val/high_school_government_and_politics_val.csv
data/val/college_physics_val.csv
data/val/miscellaneous_val.csv
data/val/machine_learning_val.csv
data/val/high_school_mathematics_val.csv
data/val/high_school_biology_val.csv
data/val/abstract_algebra_val.csv
data/val/college_medicine_val.csv
data/val/clinical_knowledge_val.csv
data/val/electrical_engineering_val.csv
data/val/econometrics_val.csv
data/val/world_religions_val.cs

57

In [31]:
def construct_message(agents, question, idx):
    if len(agents) == 0:
        return {"role": "user", "content": "Can you double check that your answer is correct. Put your final answer in the form (X) at the end of your response."}

    prefix_string = "These are the solutions to the problem from other agents: "

    for agent in agents:
        agent_response = agent[idx]["content"]
        response = "\n\n One agent solution: ```{}```".format(agent_response)

        prefix_string = prefix_string + response

    prefix_string = prefix_string + """\n\n Using the reasoning from other agents as additional advice, can you give an updated answer? Examine your solution and that other agents step by step. Put your answer in the form (X) at the end of your response.""".format(question)
    return {"role": "user", "content": prefix_string}


def construct_assistant_message(completion):
    content = completion[0]["generated_text"][-1]["content"]
    return {"role": "assistant", "content": content}


def generate_answer(agent_config, agent_context):
    completion = agent_config["pipe"](
      agent_context,
      max_new_tokens=1024,
      do_sample=True,
      temperature=agent_config["temp"],
      top_p=0.9,
    )


    return completion


def parse_question_answer(df, ix):
    question = df.iloc[ix, 0]
    a = df.iloc[ix, 1]
    b = df.iloc[ix, 2]
    c = df.iloc[ix, 3]
    d = df.iloc[ix, 4]

    question = "Can you answer the following question as accurately as possible? {}: A) {}, B) {}, C) {}, D) {} Explain your answer, putting the answer in the form (X) at the end of your response.".format(question, a, b, c, d)

    answer = df.iloc[ix, 5]

    return question, answer



In [32]:
random.seed(0)
response_dict = {}

for i in range(questions):
    df = random.choice(dfs)
    ix = len(df)
    idx = random.randint(0, ix-1)

    question, answer = parse_question_answer(df, idx)

    agent_contexts = [[{"role": "user", "content": question}] for agent in range(len(agent_configs))]
    print(f"Question {i+1}/{questions}: {question}")
    print(f"Answer: {answer}")

    for round in range(rounds):
        for i, agent_context in enumerate(agent_contexts):
            print(f"Round: {round + 1}/{rounds} | Agent: {i+1}/{len(agent_configs)}")
            if round != 0:
                agent_contexts_other = agent_contexts[:i] + agent_contexts[i+1:]
                message = construct_message(agent_contexts_other, question, 2 * round - 1)
                agent_context.append(message)

            completion = generate_answer(agent_configs[i], agent_context)

            assistant_message = construct_assistant_message(completion)
            agent_context.append(assistant_message)
            print(assistant_message)

    response_dict[question] = (agent_contexts, answer)

json.dump(response_dict, open("mmlu_{}_{}.json".format(len(agent_configs), rounds), "w"))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question 1/100: Can you answer the following question as accurately as possible? The trigger to initiate the contractile process in skeletal muscle is:: A) potassium binding to myosin., B) calcium binding to tropomyosin., C) ATP binding to the myosin cross bridges., D) calcium binding to troponin. Explain your answer, putting the answer in the form (X) at the end of your response.
Answer: D
Round: 1/2 | Agent: 1/3
{'role': 'assistant', 'content': "The correct answer is (B) calcium binding to tropomyosin.\n\nThe contractile process in skeletal muscle is initiated by the binding of calcium ions (Ca²⁺) to tropomyosin, a protein that regulates the interaction between actin and myosin filaments. When calcium ions bind to tropomyosin, it causes a conformational change that exposes the myosin binding sites on the actin filaments. This exposes the myosin heads, which then bind to the actin filaments and initiate the sliding filament mechanism, leading to muscle contraction.\n\nHere's a brief e

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': 'To accurately answer the question, I need more information about skeletal muscle\'s function and the role of the "trigger" to initiate its contractile process.\n\nThe question is asking for the trigger to initiate the contractile process in skeletal muscle. Here is a description of how a trigger mechanism may function in muscle cells:\n\n1. Calcium is one of the most abundant and essential nutrients for muscle function, playing a crucial role in muscle contraction.\n\n2. A protein named myosin, primarily involved in muscle fiber contraction, interacts with its structural protein troponin, binding to two locations named Tropomyosin II and I.\n\nWhen calcium levels are low (hypocalcemia), the high- and high-affinity states of myosin Tropomyosin and Myosin head 2 form interconversions (Troponin II-I, or "T-tub") in muscle cells. This conformation permits an open channel to allow passive transport of calcium out of the myosin-binding site (myosin-actin con

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': 'In my first attempts to find this information in an English resource with a link to any English online book, I did not encounter the solution you provided. The source in response to the problem and one from another agent did not provide the same exact wording or explanation. Thus, in the most logical direction given the information given, I had not provided an accurate answer to the question. I apologize for that. Let me do the revised response based on those available resources. If you find a reliable source with direct access to this fact in an English-language dictionary, please let me know, and I will provide a step-by-step explanation.\n\nBased on the provided solutions from agents in the study:\n\nOne agent solution is (A) "Potassium binding to myosin is not the primary trigger for the contractile process."\n\nExplanation: Potassium ions play a role in regulating muscle contraction, but they do not bind directly to tropomyosin, the key player in 

KeyboardInterrupt: 